<a href="https://colab.research.google.com/github/gandhimonil9823/Covid_Stock_Analysis/blob/main/Covid_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [130]:
!apt-get install -q libgeos-3.5.0
!apt-get install -q libgeos-dev
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip

Reading package lists...
Building dependency tree...
Reading state information...
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists...
Building dependency tree...
Reading state information...
libgeos-dev is already the newest version (3.6.2-1build2).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.


In [131]:
!pip install -q pyproj==1.9.6

In [132]:
!pip install pathpy

In [133]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import statistics 
import numpy as np
from mpl_toolkits.basemap import Basemap
%matplotlib inline
import pathpy as pp
from geopy import distance
from geopy.distance import geodesic
import itertools
import math

In [134]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [135]:
# Merging the Stock Indexes and the currency data with rows as dates and columns as index values
# Preprocessing the data sets with online steps for data processing 
# Figuring a way to convert the unemployment data into county wise data - probably using a probability distribution
# Deal with missing stock data for weekends and holidays

Stock Data: Pre-processing


In [136]:
# Reading CSVs for three stock indices
df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/DJI.csv")
df_DJI = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/NASDAQ.csv")
df_NASDAQ = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/S&P500.csv")
df_SP500 = df[["Date", "Adj Close", "Volume"]]

df_covid_clean_complete = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/covid_19_clean_complete.csv")

trend_seasonality_df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/trend_seasonality_covid.csv")


df_centroid = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/pre_processed_centroid.csv")

df_with_missing = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/stock_closing_prices_188.csv")
df_with_missing_historical = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/historical_data_filled_188.csv")

In [ ]:
# Rename columns in each index in order to merge the indices 
df_NASDAQ = df_NASDAQ.rename(columns={'Adj Close': 'Adj_Close_NASDAQ', 'Volume': 'Volume_NASDAQ'})
df_SP500 = df_SP500.rename(columns={'Adj Close': 'Adj_Close_SP500', 'Volume': 'Volume_SP500'})
df_DJI = df_DJI.rename(columns={'Adj Close': 'Adj_Close_DJI', 'Volume': 'Volume_DJI'})

In [ ]:
# Merging data of three stock indices based on column: 'Date' (merging two DFs at once)
df_Merged = pd.merge(df_SP500, df_DJI, on=['Date'])
df_stock_data = pd.merge(df_Merged, df_NASDAQ, on=['Date'])

In [ ]:
df_stock_data.head()

In [ ]:
df_stock_data.shape

In [ ]:
df_with_missing.head()

In [ ]:
df_with_missing.shape

In [ ]:
df_with_missing_historical

# COVID-19 Data Loading and Preprocessing

In [ ]:
# Function to load the covid_clean_complete dataset from the drive
def load_dataset_us_cases():  
  df_covid_clean_complete = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/covid_19_clean_complete.csv")
  return df_covid_clean_complete

# Function to preprocess the clean_complete_dataset
def preprocess_us_cases_df():
  df_US_daily_cases = load_dataset_us_cases()

  # get the daily data only for US
  df_US_daily_cases = df_covid_clean_complete.loc[df_covid_clean_complete['Country/Region'] == "US"]
  
  df_US_daily_cases = df_US_daily_cases.reset_index(drop=True)

  # Added a column for new_cases daily by getting the differerence of the confirmed cases
  df_US_daily_cases["New_cases"] = df_US_daily_cases["Confirmed"].diff()
  
  df_US_daily_cases['New_cases'] = df_US_daily_cases['New_cases'].fillna(0)

  df_US_daily_cases = df_US_daily_cases.rename(columns={"Confirmed": "Total_Confirmed", "Deaths" : "Total_Deaths"})

  return df_US_daily_cases


# Standardize the COVID variables like New_cases, Total_cases, Total_deaths

In [ ]:
# Function to standardize the covid_variables like New_cases, Total_cases, Total_deaths
# Adds new columns to the df with the standarized values
# params - the dataframe, and the column name (COVID Variable) which has to be standarized
def covid_variable_impact_cal(dataframe_to_standardize, column_name):
  # col_mean = dataframe_to_standardize[column_name].mean()
  # col_std  = dataframe_to_standardize[column_name].std()

  col_name = "Standardized_Value_" + column_name
  dataframe_to_standardize[col_name] = dataframe_to_standardize[column_name]

  i = 0
  j = 0
  while i < len(dataframe_to_standardize):
    j = i
    if i > 7:
      col_mean = dataframe_to_standardize[column_name][j - 8: j - 1].mean()
      col_std  = statistics.stdev(dataframe_to_standardize[column_name][j - 8: j - 1])

      curr_daily_value = dataframe_to_standardize[column_name][i]

      formula_cal_value = (curr_daily_value - col_mean) / col_std

      dataframe_to_standardize.loc[i, col_name] = formula_cal_value

    i += 1
  dataframe_to_standardize[col_name].replace(np.inf, 0, inplace=True)
  return dataframe_to_standardize

In [ ]:
df_US_daily_cases = preprocess_us_cases_df()
df_US_daily_cases.head()



In [ ]:
df_standarized_total_confirmed = covid_variable_impact_cal(df_US_daily_cases, "Total_Confirmed")

df_standarized_total_confirmed.reset_index().plot(x='index', y='Standardized_Value_Total_Confirmed')    

df_standarized_Total_Deaths = covid_variable_impact_cal(df_US_daily_cases, "Total_Deaths")

df_standarized_Total_Deaths.reset_index().plot(x='index', y='Standardized_Value_Total_Deaths')

df_standarized_New_cases = covid_variable_impact_cal(df_US_daily_cases, "New_cases")

df_standarized_New_cases.reset_index().plot(x='index', y='Standardized_Value_New_cases')

# Complex Network Analysis

In [ ]:
# Using Centroid of all the long and lat
# def combinations_distance_df(df_original):
#     counties  = df_original["Combined_Key"]
    
#     list_of_counties_combinations = list(itertools.combinations(counties,2))

#     df_to_return = pd.DataFrame({"Combinations" : list_of_counties_combinations})

#     array_distances = []
#     for item in list_of_counties_combinations:
#         row_element1 = df_original.loc[df_original['Combined_Key'] == item[0]]
#         row_element2 = df_original.loc[df_original['Combined_Key'] == item[1]]
        
#         value1 = float(row_element1["Centroid_Distance"].tolist()[0])
#         value2 = float(row_element2["Centroid_Distance"].tolist()[0])

#         array_distances.append(abs(value1 - value2))

#     df_to_return["Distance"] = array_distances
#     return df_to_return    


# Using Two counties difference directly
def combinations_distance_df(df_original):
    counties  = df_original["Combined_Key"]

    list_of_counties_combinations = list(itertools.combinations(counties,2))

    df_to_return = pd.DataFrame({"Combinations" : list_of_counties_combinations})

    array_distances = []
    for item in list_of_counties_combinations:
        row_element1 = df_original.loc[df_original['Combined_Key'] == item[0]]
        row_element2 = df_original.loc[df_original['Combined_Key'] == item[1]]

        long1 = float(row_element1["Long_"].tolist()[0])
        lat1 = float(row_element1["Lat"].tolist()[0])
        
        long2 = float(row_element2["Long_"].tolist()[0])
        lat2 = float(row_element2["Lat"].tolist()[0])

        point_n = geodesic((lat1, long1),(lat2, long2)).miles
        
        array_distances.append(point_n)

    df_to_return["Distance"] = array_distances
    return df_to_return    

    

In [ ]:
df_centroid = df_centroid[df_centroid.New_cases != 0]

df_centroid = df_centroid.reset_index(drop=True)

groupby_daily = df_centroid.groupby(df_centroid["Date"])

i = 0
g = None
for name, group in groupby_daily:
# 2174
    if len(group) == 2174:
        fig = plt.figure(figsize=(16, 10))
        m_ = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
        m_.readshapefile('/content/drive/Shared drives/Stock_Covid_Data/COVID_19/st99_d00', name='states')
        df_comb_counties_distance = combinations_distance_df(group)
        G_t = pp.Network(directed=False)
        for index_group, row_group in group.iterrows():
            if row_group["New_cases"] >= 0:
                G_t.add_node(str(row_group["Combined_Key"]))
        m_.scatter(group["Long_"].tolist(), group["Lat"].tolist(), latlon=True, s=25, c="red", marker='o')    
        for index, row in df_comb_counties_distance.iterrows():
            if row["Distance"] < 100:
                tuple_county = row["Combinations"]
                if str(tuple_county[0]) in G_t.nodes and str(tuple_county[1]) in G_t.nodes:
                    G_t.add_edge(str(tuple_county[0]), str(tuple_county[1]))
        plt.title(str(name), fontsize=20)
        
        g = G_t
#         print(name)


#         pp.visualisation.plot(G_t)    
    # print(len(group))
        
style = {'width': 1000, 
          'height': 1000
        }
pp.visualisation.export_html(g, 'my_temporal_network.html')
pp.visualisation.plot(g,**style)

# Complex Network Analysis for Trend and Seasonality


In [137]:
# trend_seasonality_df = trend_seasonality_df[trend_seasonality_df["Date"] == "2020-7-27"]

# print(len(trend_seasonality_df["Trend"]))
# print(trend_seasonality_df["Trend"].max())

# print(trend_seasonality_df["Seasonal"].min())
# print(trend_seasonality_df["Seasonal"].max())


In [138]:
# trend_seasonality_df['bin'] = pd.qcut(trend_seasonality_df['Trend'], q=10
#                                       )

# # print(trend_seasonality_df)

# groupby_bin = trend_seasonality_df.groupby(trend_seasonality_df["bin"])

# for name, group in groupby_bin:
#   print(len(group))

# # array = trend_seasonality_df["bin"].unique()

# # array[0]




In [139]:
def combinations_bin_counties(df_original):
    counties  = df_original["Combined_Key"]

    list_of_counties_combinations = list(itertools.combinations(counties,2))

    df_to_return = pd.DataFrame({"Combinations" : list_of_counties_combinations})

    return df_to_return

In [153]:
# trend_seasonality_df = trend_seasonality_df[trend_seasonality_df.Trend != 0]

# trend_seasonality_df = trend_seasonality_df.reset_index(drop=True)

groupby_daily = trend_seasonality_df.groupby(trend_seasonality_df["Date"])
g = None
for name, group in groupby_daily: 
  if name == "2020-4-15" :
    G_t = pp.Network(directed=False)
    group['bin'] = pd.qcut(group['Trend'], q=30, duplicates='drop')
    groupby_bin = group.groupby(group["bin"])
    for name_bin, group_bin in groupby_bin:
      df_comb_counties_bin = combinations_bin_counties(group_bin)
      for index_group, row_group in group_bin.iterrows():
        if row_group["New_cases"] >= 10:
          G_t.add_node(str(row_group["Combined_Key"]))
      for index_bin, row_bin in df_comb_counties_bin.iterrows():
        tuple_county = row_bin["Combinations"]
        if str(tuple_county[0]) in G_t.nodes and str(tuple_county[1]) in G_t.nodes:
          G_t.add_edge(str(tuple_county[0]), str(tuple_county[1]))      
    g = G_t



style = {'width': 1000, 
          'height': 1000
        }
pp.visualisation.export_html(g, 'my_temporal_network.html')
pp.visualisation.plot(g,**style)



## Computing Stock paramaters - Abnormal price and Volatality:

In [ ]:
def compute_stock_params(covid_stock_values, precovid_stock_values):
  
  # Average of daily stock prices in the pre-COVID period (P_t-i)
  precovid_stocks_sum = precovid_stock_values[1::].sum()
  avg_precovid_stocks = precovid_stocks_sum[1::].div(188)

  # Differece between the daily stock prices in the COVID period and the average of daily stock prices in the pre-covid period (P_t - P_t-i)
  snp_diff = [daily_stocks - avg_precovid_stocks[0] for daily_stocks in precovid_stock_values['Adj_Close_SP500']]
  dji_diff = [daily_stocks - avg_precovid_stocks[1] for daily_stocks in precovid_stock_values['Adj_Close_DJI']]
  nasdaq_diff = [daily_stocks - avg_precovid_stocks[2] for daily_stocks in precovid_stock_values['Adj_Close_NASDAQ']]

  diff = pd.DataFrame(
    {'Adj_Close_SP500': snp_diff,
     'Adj_Close_DJI': dji_diff,
     'Adj_Close_NASDAQ': nasdaq_diff
    })
  
  # Standard deviatoon of the pre-COVID period
  sd_stock_indices = (diff**2 / 188)**0.5

  # Abnormal price  
  abnormal_price = (covid_stock_values -  avg_precovid_stocks) / sd_stock_indices;
  abnormal_price['Date'] = covid_stock_values['Date']

  print("covid_stock_values = ",covid_stock_values)
  # log(P_t/P_t-1)
  rt = pd.DataFrame([[0, 0, 0]], columns = ['Adj_Close_SP500', 'Adj_Close_DJI', 'Adj_Close_NASDAQ'])
  for i in range(1, len(covid_stock_values)):
    previous = covid_stock_values.loc[i-1,covid_stock_values.columns != 'Date']
    today = covid_stock_values.loc[i,covid_stock_values.columns != 'Date']
    temp = today.div(previous)
    rt = rt.append(temp, ignore_index=True)

  rt = rt.apply(np.log10)
  
  # Volatality of closing stock prices in the COVID period
  vol = rt**2

  rt['Date'] = covid_stock_values['Date']
  vol['Date'] = covid_stock_values['Date']
  
  return abnormal_price, rt, vol

In [ ]:
abnormal_price, rt, vol = compute_stock_params(df_with_missing, df_with_missing_historical)
abnormal_price

In [ ]:
rt

In [ ]:
vol

In [ ]:
import warnings 
warnings.filterwarnings('ignore')
import statsmodels.api as sm

In [ ]:
df_snp500 = df_with_missing[['Date','Adj_Close_SP500']]

In [ ]:
df_snp500.head()

In [ ]:
df_snp500.set_index('Date', inplace = True)
df_snp500['Adj_Close_SP500'] = df_snp500['Adj_Close_SP500'].astype(float)

In [ ]:
df_snp500.describe()

In [ ]:
result = seasonal_decompose(df_snp500['Adj_Close_SP500'], model='additive', freq=25)

fig = plt.figure()  
fig = result.plot()  
fig.set_size_inches(10, 7)

In [ ]:
df_snp500.plot(figsize=(10, 3))

In [ ]:
from statsmodels.tsa.stattools import adfuller
def test_stationarity(timeseries, window = 25, cutoff = 0.01):

    #Determing rolling statistics
    rolmean = timeseries.rolling(window).mean()
    rolstd = timeseries.rolling(window).std()

    #Plot rolling statistics:
    fig = plt.figure(figsize=(12, 8))
    orig = plt.plot(timeseries.values, color='blue',label='Original')
    mean = plt.plot(rolmean.values, color='red', label='Rolling Mean')
    std = plt.plot(rolstd.values, color='black', label = 'Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean & Standard Deviation')
    plt.show()
    
    #Perform Dickey-Fuller test:
    print('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', maxlag = 20 )
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    pvalue = dftest[1]
    if pvalue < cutoff:
        print('p-value = %.4f. The series is likely stationary.' % pvalue)
    else:
        print('p-value = %.4f. The series is likely non-stationary.' % pvalue)
    
    print(dfoutput)

In [ ]:
test_stationarity(df_snp500['Adj_Close_SP500'])

In [ ]:
df_snp500['SP_Third_Difference'] = df_snp500['Adj_Close_SP500'] - df_snp500['Adj_Close_SP500'].shift(3)
test_stationarity(df_snp500['SP_Third_Difference'].dropna(), window = 25)

In [ ]:
import statsmodels.api as sm

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_snp500['Adj_Close_SP500'], lags=40, ax=ax1) # 
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_snp500['Adj_Close_SP500'], lags=40, ax=ax2)

In [ ]:
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(df_snp500['SP_Third_Difference'].dropna(), lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(df_snp500['SP_Third_Difference'].dropna(), lags=40, ax=ax2)

In [ ]:
arima_mod6 = sm.tsa.ARIMA(df_snp500['Adj_Close_SP500'], (3,3,5)).fit(disp=False)
print(arima_mod6.summary())

In [ ]:
# model = ARIMA(df_snp500['Adj_Close_SP500'], order=(2, 1, 2))  
# results_ARIMA = model.fit()  


df_snp500['forecast']=arima_mod6.predict(start=160,end=187,dynamic=True)
df_snp500[['Adj_Close_SP500','forecast']].plot(figsize=(12,8))


# plt.plot(df_snp500['SP_First_Difference'].values)
# plt.plot(results_ARIMA.fittedvalues, color='red')
# plt.title('RSS: %.4f'% sum((results_ARIMA.fittedvalues-ts_log_diff)**2))

In [ ]:
from scipy import stats
from scipy.stats import normaltest

resid = arima_mod6.resid
print(normaltest(resid))
# returns a 2-tuple of the chi-squared statistic, and the associated p-value. the p-value is very small, meaning
# the residual is not a normal distribution

fig = plt.figure(figsize=(12,8))
ax0 = fig.add_subplot(111)

sns.distplot(resid ,fit = stats.norm, ax = ax0) # need to import scipy.stats

# Get the fitted parameters used by the function
(mu, sigma) = stats.norm.fit(resid)

#Now plot the distribution using 
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('Residual distribution')


# ACF and PACF
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(arima_mod6.resid, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(arima_mod6.resid, lags=40, ax=ax2)

In [ ]:
# sarima_mod6 = sm.tsa.statespace.SARIMAX(df_snp500['Adj_Close_SP500'], trend='n', order=(2,1,2)).fit()
# print(sarima_mod6.summary())

sarimax_model=sm.tsa.statespace.SARIMAX(df_snp500['Adj_Close_SP500'],order=(2, 1, 1),seasonal_order=(2,1,1,32)).fit()
print(sarimax_model.summary())


In [ ]:
df_snp500['forecast']=sarimax_model.predict(start=150,end=187,dynamic=True)
df_snp500[['Adj_Close_SP500','forecast']].plot(figsize=(12,8))

In [ ]:
resid = sarima_mod6.resid
print(normaltest(resid))

fig = plt.figure(figsize=(12,8))
ax0 = fig.add_subplot(111)

sns.distplot(resid ,fit = stats.norm, ax = ax0) # need to import scipy.stats

# Get the fitted parameters used by the function
(mu, sigma) = stats.norm.fit(resid)

#Now plot the distribution using 
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)], loc='best')
plt.ylabel('Frequency')
plt.title('Residual distribution')


# ACF and PACF
fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(arima_mod6.resid, lags=40, ax=ax1)
ax2 = fig.add_subplot(212)
fig = sm.graphics.tsa.plot_pacf(arima_mod6.resid, lags=40, ax=ax2)